# Transformação para formato Time Series

## Pacotes necessários

In [1]:
import pandas as pd
import os

## Leitura dos dados

In [ ]:
# Carregar o dataframe recodificado
df = pd.read_parquet("/home/usuario/Documentos/Escorpiao_series/Dados_processados/df1_recodificado.parquet")

# Filtrar os dados para incluir apenas os anos de 2015 a 2023
df = df[df['ANO'].isin(range(2015, 2024))]

## Agrupamento e calculo da Tx Incidencia

In [ ]:
# Agrupar os dados por município, ano e população, contando o número de casos
dfg = df.groupby(['NOME_MUNI', 'ANO', 'POPULACAO']).size().reset_index(name='CASOS')

# Calcular a taxa de incidência por 100.000 habitantes
dfg['TAXA_INCIDENCIA'] = (dfg['CASOS'] / dfg['POPULACAO']) * 100000

# Arredondar a taxa de incidência para uma casa decimal
dfg['TAXA_INCIDENCIA'] = dfg['TAXA_INCIDENCIA'].round(1)

## Criar um dataframe por municipio

In [ ]:
municipios = dfg['NOME_MUNI'].unique()
series_por_municipio = {}

for municipio in municipios:
    # Criar uma cópia selecionada para evitar SettingWithCopyWarning. Sem o copy, o objeto é so uma view do dataframe original, com o copy, é um novo objeto independente.    
    df_mun = dfg[dfg['NOME_MUNI'] == municipio].copy() 
    df_mun = df_mun.sort_values('ANO') # organizar por ano
    df_mun.set_index('ANO', inplace=True) # definir o ano como índice (apropriado para séries temporais)
    series_por_municipio[municipio] = df_mun[['TAXA_INCIDENCIA']] # manter apenas a coluna de interesse

In [18]:
series_por_municipio['AGUAI']

,TAXA_INCIDENCIA
ANO,
2015,30.4
2016,24.3
2017,51.7
2018,57.8
2019,103.4
